In [1]:
import pandas as pd

#Reading the Previously Saved Dataframe (Scraped from Wikipedia) With Added GeoLocation
Canada_data=pd.read_csv('Canada_data_geo.csv',sep=';')
Canada_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
toronto_data = Canada_data[Canada_data['Borough'] .str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [4]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [5]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [6]:
CLIENT_ID = 'JPE1B3LCFAZNIPBYYKCSVYW0J0DDIFHXDBOORUHMWWEXIK1E' # your Foursquare ID
CLIENT_SECRET = 'DGKXL4PISPPEYGZBXORRS52CFOHIZXBSX1R5CAEKMENX4IFI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JPE1B3LCFAZNIPBYYKCSVYW0J0DDIFHXDBOORUHMWWEXIK1E
CLIENT_SECRET:DGKXL4PISPPEYGZBXORRS52CFOHIZXBSX1R5CAEKMENX4IFI


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=20):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    ########################    can be expanded into 
    #l=[]

#for venue_list in venues_list:
 #   for item in venue_list:
  #      l.append(item)
    
#nearby_venues = pd.DataFrame(l)
 ###########################################################   
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [9]:
print(toronto_venues.shape)
toronto_venues.head()

(647, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [10]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"Brockton, Parkdale Village, Exhibition Place",20,20,20,20,20,20
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,20,20,20,20,20,20
Christie,17,17,17,17,17,17
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
Davisville,20,20,20,20,20,20


In [11]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 173 uniques categories.


In [15]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.shape

(647, 173)

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot = toronto_onehot[ ['Neighborhood'] + [ col for col in toronto_onehot.columns if col != 'Neighborhood' ] ]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.05,0.05,0.00,0.00,0.05,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.05,0.000000,0.00,0.00,0.000000,0.05,0.050000,0.00,0.00,0.000000,0.05,0.00,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.050000,0.000000,0.00,0.00,0.00,0.000,0.00,0.05,0.05,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.050000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.05,0.000000,0.00,0.000000,0.00,0.00,0.050000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.10,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.050000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.00,0.050000,0.000000,0.00,0.150000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.05,0.000000,0

In [20]:
toronto_grouped.shape

(39, 173)

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Bistro,Bakery,Park,Cocktail Bar,Concert Hall,Museum,Cheese Shop,Liquor Store,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Gym,Furniture / Home Store,Italian Restaurant,Nightclub,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique
4,Central Bay Street,Coffee Shop,Sushi Restaurant,Chinese Restaurant,Modern European Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Bubble Tea Shop,Sandwich Place,Park,Spa


In [24]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 2, 0, 0, 0, 0, 0], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Breakfast Spot,Chocolate Shop,Bakery,Pub,Restaurant,Dessert Shop,Distribution Center,Farmers Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Hobby Shop,Park,Creperie,Mexican Restaurant,Burrito Place,Burger Joint,Diner,Discount Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Burrito Place,Sporting Goods Shop,Electronics Store,Ramen Restaurant,Steakhouse,Clothing Store,Plaza,Mexican Restaurant,Tea Room
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Restaurant,Coffee Shop,BBQ Joint,Café,Hotel,Food Truck,Italian Restaurant,Japanese Restaurant,Poke Place
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Trail,Health Food Store,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Café,Burrito Place,Sporting Goods Shop,Electronics Store,Ramen Restaurant,Steakhouse,Clothing Store,Plaza,Mexican Restaurant,Tea Room
3,Downtown Toronto,0,Gastropub,Restaurant,Coffee Shop,BBQ Joint,Café,Hotel,Food Truck,Italian Restaurant,Japanese Restaurant,Poke Place
5,Downtown Toronto,0,Seafood Restaurant,Bistro,Bakery,Park,Cocktail Bar,Concert Hall,Museum,Cheese Shop,Liquor Store,Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Candy Store,Restaurant,Diner,Italian Restaurant,Baby Store
9,West Toronto,0,Pharmacy,Bakery,Grocery Store,Supermarket,Music Venue,Middle Eastern Restaurant,Portuguese Restaurant,Café,Brewery,Brazilian Restaurant
10,Downtown Toronto,0,Park,Plaza,Hotel,Supermarket,Skating Rink,Dessert Shop,Japanese Restaurant,Bubble Tea Shop,Sporting Goods Shop,Salad Place
11,West Toronto,0,Bar,Yoga Studio,Asian Restaurant,Greek Restaurant,French Restaurant,Ice Cream Shop,Japanese Restaurant,Korean Restaurant,Cuban Restaurant,New American Restaurant
12,East Toronto,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore,Fruit & Vegetable Store,Juice Bar,Dessert Shop,Cosmetics Shop,Pub
13,Downtown Toronto,0,Coffee Shop,Café,Deli / Bodega,Bakery,Steakhouse,Pub,Tea Room,Restaurant,Beer Bar,Gym / Fitness Center
14,West Toronto,0,Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Gym,Furniture / Home Store,Italian Restaurant,Nightclub,Performing Arts Venue


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
33,Downtown Toronto,1,Park,Playground,Trail,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Park,Breakfast Spot,Chocolate Shop,Bakery,Pub,Restaurant,Dessert Shop,Distribution Center,Farmers Market
1,Downtown Toronto,2,Coffee Shop,Sushi Restaurant,Hobby Shop,Park,Creperie,Mexican Restaurant,Burrito Place,Burger Joint,Diner,Discount Store
4,East Toronto,2,Pub,Trail,Health Food Store,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
6,Downtown Toronto,2,Coffee Shop,Sushi Restaurant,Chinese Restaurant,Modern European Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Bubble Tea Shop,Sandwich Place,Park,Spa
8,Downtown Toronto,2,Coffee Shop,Opera House,Concert Hall,Restaurant,Smoke Shop,Speakeasy,Hotel,Lounge,Steakhouse,Plaza
24,Central Toronto,2,Café,Sandwich Place,Coffee Shop,Pizza Place,History Museum,Pharmacy,Pub,Middle Eastern Restaurant,Burger Joint,Liquor Store
28,West Toronto,2,Italian Restaurant,Sushi Restaurant,Pub,Gourmet Shop,Bookstore,Latin American Restaurant,Coffee Shop,Smoothie Shop,Indie Movie Theater,Falafel Restaurant
31,Central Toronto,2,Pub,Coffee Shop,Sushi Restaurant,Sandwich Place,Restaurant,Bank,Sports Bar,Light Rail Station,Bagel Shop,Liquor Store
32,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Deli / Bodega,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Ice Cream Shop,Garden,Home Service,Yoga Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,5,Park,Restaurant,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


As Four of the six categories contain only one entry we will reduce the no. of categories to 3 instead hoping for a more coherent clustering output

In [42]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels']= kmeans.labels_

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

[1 1 0 1 1 1 1 1 1 1]


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Chocolate Shop,Bakery,Pub,Restaurant,Dessert Shop,Distribution Center,Farmers Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Hobby Shop,Park,Creperie,Mexican Restaurant,Burrito Place,Burger Joint,Diner,Discount Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Burrito Place,Sporting Goods Shop,Electronics Store,Ramen Restaurant,Steakhouse,Clothing Store,Plaza,Mexican Restaurant,Tea Room
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Gastropub,Restaurant,Coffee Shop,BBQ Joint,Café,Hotel,Food Truck,Italian Restaurant,Japanese Restaurant,Poke Place
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,0,Pub,Trail,Health Food Store,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
10,Downtown Toronto,0,Park,Plaza,Hotel,Supermarket,Skating Rink,Dessert Shop,Japanese Restaurant,Bubble Tea Shop,Sporting Goods Shop,Salad Place
11,West Toronto,0,Bar,Yoga Studio,Asian Restaurant,Greek Restaurant,French Restaurant,Ice Cream Shop,Japanese Restaurant,Korean Restaurant,Cuban Restaurant,New American Restaurant
12,East Toronto,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Bookstore,Fruit & Vegetable Store,Juice Bar,Dessert Shop,Cosmetics Shop,Pub
15,East Toronto,0,Fast Food Restaurant,Park,Italian Restaurant,Steakhouse,Pub,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Restaurant,Light Rail Station
19,Central Toronto,0,Ice Cream Shop,Garden,Home Service,Yoga Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
21,Central Toronto,0,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Deli / Bodega,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
38,East Toronto,0,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Recording Studio


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Breakfast Spot,Chocolate Shop,Bakery,Pub,Restaurant,Dessert Shop,Distribution Center,Farmers Market
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Hobby Shop,Park,Creperie,Mexican Restaurant,Burrito Place,Burger Joint,Diner,Discount Store
2,Downtown Toronto,1,Café,Burrito Place,Sporting Goods Shop,Electronics Store,Ramen Restaurant,Steakhouse,Clothing Store,Plaza,Mexican Restaurant,Tea Room
3,Downtown Toronto,1,Gastropub,Restaurant,Coffee Shop,BBQ Joint,Café,Hotel,Food Truck,Italian Restaurant,Japanese Restaurant,Poke Place
5,Downtown Toronto,1,Seafood Restaurant,Bistro,Bakery,Park,Cocktail Bar,Concert Hall,Museum,Cheese Shop,Liquor Store,Restaurant
6,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Chinese Restaurant,Modern European Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Bubble Tea Shop,Sandwich Place,Park,Spa
7,Downtown Toronto,1,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Candy Store,Restaurant,Diner,Italian Restaurant,Baby Store
8,Downtown Toronto,1,Coffee Shop,Opera House,Concert Hall,Restaurant,Smoke Shop,Speakeasy,Hotel,Lounge,Steakhouse,Plaza
9,West Toronto,1,Pharmacy,Bakery,Grocery Store,Supermarket,Music Venue,Middle Eastern Restaurant,Portuguese Restaurant,Café,Brewery,Brazilian Restaurant
13,Downtown Toronto,1,Coffee Shop,Café,Deli / Bodega,Bakery,Steakhouse,Pub,Tea Room,Restaurant,Beer Bar,Gym / Fitness Center


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
29,Central Toronto,2,Park,Restaurant,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
33,Downtown Toronto,2,Park,Playground,Trail,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
